In [1]:
import os
import numpy as np
import rasterio
import math
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from rasterio.transform import rowcol
from joblib import Parallel, delayed


In [2]:
pasta_principal = './Dados/Clima/'

pastas = [nome for nome in os.listdir(pasta_principal) 
          if os.path.isdir(os.path.join(pasta_principal, nome))]
pastas

['Precipitação', 'Temperatura Máxima', 'Temperatura Mínima']

In [3]:
def ajustarNumeros(arr):
    return np.where(
            arr >= 100,
            np.round(arr / 10, 2),
            np.round(arr, 2)
        )

In [4]:
# # Função para salvar o arquivo de saída

def salvarArquivo(dados, profile, caminho, nome):
    os.makedirs(caminho, exist_ok=True)
    saida = os.path.join(caminho, f"{nome}.tif")
    profile.update(dtype='float32', nodata=0)
    with rasterio.open(saida, 'w', **profile) as dst:
        dst.write(dados, 1)

def processarImagens(caminhos, pasta):
    profile = None    
    max_hist = None
    min_hist = None
    soma = None
    print(pasta)
    for caminho in caminhos:
        with rasterio.open(caminho) as src:
            img = ajustarNumeros(src.read(1))
            if profile is None:
                profile = src.profile
              
            if max_hist is None:
                max_hist = img.copy()
                min_hist = img.copy()
                soma = img.astype('float64')
            else:
                max_hist = np.fmax(max_hist, img)
                min_hist = np.fmin(min_hist, img)
                soma += img
                
    avg_hist = soma/len(caminhos)    
    salvarArquivo(max_hist.astype('float32'), profile, pasta, "historico_maximo")
    print(f"Máximo histórico salvo em {pasta}")
  
    salvarArquivo(min_hist.astype('float32'), profile, pasta, "historico_minimo")
    print(f"Mínimo histórico salvo em {pasta}")
  
  
    salvarArquivo(avg_hist.astype('float32'), profile, pasta, "historico_media")
    print(f"Média histórica salvo em {pasta}")
            
            
            
def processarPastas():
    with ThreadPoolExecutor() as executor:
        futures = []
        for pasta in pastas:
            caminhos = [os.path.join(f"{pasta_principal}/{pasta}", f)
            for f in os.listdir(f"{pasta_principal}/{pasta}") if f.endswith('.tif')]
            futures.append(executor.submit(processarImagens, caminhos, pasta))
        for future in futures:
            future.result()

processarPastas()

Precipitação
Temperatura Máxima
Temperatura Mínima
Máximo histórico salvo em Precipitação
Mínimo histórico salvo em Precipitação
Média histórica salvo em Precipitação
Máximo histórico salvo em Temperatura Máxima
Máximo histórico salvo em Temperatura Mínima
Mínimo histórico salvo em Temperatura Máxima
Mínimo histórico salvo em Temperatura Mínima
Média histórica salvo em Temperatura Máxima
Média histórica salvo em Temperatura Mínima


In [5]:
with rasterio.open(f"{pasta_principal}/{pastas[1]}/historico_minimo.tif") as src: 
    valores = src.read(1)
    

In [2]:
df = pd.read_csv("./Dados/coordenadas_com_indices.csv", index_col=0)
df = df.dropna()

df

,Latitude,Longitude,NDVI,NDWI
83946,-13.71,-73.78,0.517695,-0.521197
83947,-13.70,-73.78,0.534019,-0.532493
83948,-13.69,-73.78,0.591257,-0.564566
83949,-13.68,-73.78,0.632028,-0.594361
83950,-13.67,-73.78,0.648493,-0.592593
...,...,...,...,...
15295100,-2.13,-34.80,0.000000,0.000000
15295101,-2.12,-34.80,0.000000,0.000000
15295102,-2.11,-34.80,0.000000,0.000000
15295103,-2.10,-34.80,0.000000,0.000000


In [ ]:
tiff_path = f'{pasta_principal}/Temperatura Máxima/historico_media.tif'

# Criar lista de coordenadas (no formato lon, lat para rasterio)
coords = [(lon, lat) for lat, lon in zip(df['Latitude'], df['Longitude'])]

# Abrir o TIFF uma vez e coletar os valores
with rasterio.open(tiff_path) as src:
    maxmedia = [val[0] for val in tqdm(src.sample(coords), total=len(coords), desc="Lendo pixels")]



Lendo pixels: 100%|██████████| 5181712/5181712 [05:47<00:00, 14899.40it/s]


In [ ]:
tiff_path = f'{pasta_principal}/Temperatura Mínima/historico_media.tif'


# Criar lista de coordenadas (no formato lon, lat para rasterio)
coords = [(lon, lat) for lat, lon in zip(df['Latitude'], df['Longitude'])]

# Abrir o TIFF uma vez e coletar os valores
with rasterio.open(tiff_path) as src:
    minmedia = [val[0] for val in tqdm(src.sample(coords), total=len(coords), desc="Lendo pixels")]


Lendo pixels: 100%|██████████| 5181712/5181712 [05:47<00:00, 14931.78it/s]


In [14]:
df['TempMédia'] = (df['Max-Media'] + df['Min-Media'])/2

In [15]:
tiff_path = f'{pasta_principal}/Precipitação/historico_media.tif'


# Criar lista de coordenadas (no formato lon, lat para rasterio)
coords = [(lon, lat) for lat, lon in zip(df['Latitude'], df['Longitude'])]

# Abrir o TIFF uma vez e coletar os valores
with rasterio.open(tiff_path) as src:
    valores = [val[0] for val in tqdm(src.sample(coords), total=len(coords), desc="Lendo pixels")]

df['Precipitacao'] = valores

Lendo pixels: 100%|██████████| 5181712/5181712 [05:48<00:00, 14868.47it/s]


In [ ]:
df.head()
df.dropna()

df.to_csv("./Dados/coordenadas_com_indices_sem_nan.csv")